In [1]:
import pandas as pd

# df = pd.read_excel("data/out/Prescription.xlsx")

# 1 Nettoyage DCI 

## Étape 1 — Normalisation de base

In [95]:
import numpy as np

df["DCI"] = (
    df["DCI"]
      .astype(str)                # on force en string pour capter "NA", "nan", etc.
      .str.strip()
      .str.upper()
      .replace(
          {
              "NAN": np.nan,
              "NA": np.nan,
              "NULL": np.nan,
              "NONE": np.nan,
              "": np.nan
          }
      )
      .str.replace("’", "'", regex=False)
      .str.replace("-", " ", regex=False)
)


In [27]:
def normalize_text(s):
    return (
        s.upper()
         .strip()
         .replace("’", "'")
         .replace("-", " ")
    )


In [70]:
df["DCI_NORM"] = df["DCI"].astype(str).map(normalize_text)


## Étape 2 — Liste officielle des variations chimiques

In [ ]:
# Cette liste vient des bases ANSM / EMA / WHO

In [96]:
SALTS = [
    "SULFATE", "SULFATE DE", "CHLORHYDRATE", "CHLORHYDRATE DE",
    "PHOSPHATE", "PHOSPHATE DE",
    "ACETATE", "ACETATE DE",
    "BROMURE", "BROMURE DE",
    "IODURE", "IODURE DE",
    "NITRATE", "NITRATE DE",
    "TARTRATE", "TARTRATE DE",
    "MESILATE", "MESYLATE",
    "FUMARATE", "FUMARATE DE",
    "CITRATE", "CITRATE DE",
    "SODIQUE", "POTASSIQUE", "CALCIQUE",
    "MONOHYDRATE", "DIHYDRATE", "TRIHYDRATE",
]


## Étape 3 — Supprimer les formes chimiques

In [97]:
import re

pattern = r"\b(" + "|".join(SALTS) + r")\b\s*(DE|D')?\s*"

df["DCI_MERE"] = (
    df["DCI_NORM"]
      .str.replace(pattern, "", regex=True)
      .str.replace(r"\s{2,}", " ", regex=True)
      .str.strip()
)


## Étape 4 — Cas inversés

In [98]:
df["DCI_MERE"] = (
    df["DCI_MERE"]
      .str.replace(r"\b(" + "|".join(SALTS) + r")$", "", regex=True)
      .str.strip()
)


In [75]:
df.head()

,ID,Date,Prelibt,CIP,Prepost,DCI,ATC,DOSE,Freq,Durée,dosage,dosage_nature,cip7,Med,DCI_NORM,DCI_MERE,PRELIBT_NORM,PRELIBT_BASE
0,1,2004-04-14,BACTRIM cp Ad,3001069,"1 Comprimé(s), 1 fois / jour pendant 30 jour(s)",SULFAMETHOXAZOLE; TRIMETHOPRIME,NaN,1 Comprimé,1 fois / jour,30 jour(s),400 mg; 80 mg,un comprimé,3001069.0,BACTRIM,SULFAMETHOXAZOLE; TRIMETHOPRIME,SULFAMETHOXAZOLE; TRIMETHOPRIME,BACTRIM CP AD,BACTRIM CP AD
1,1,2004-04-14,KALETRA caps,3566794,"3 Capsule(s), Toutes les 12 heures pendant 30 ...",LOPINAVIR; RITONAVIR,J05AR10,3 Capsule,Toutes les 12 heures,30 jour(s),NaN,NaN,NaN,KALETRA,LOPINAVIR; RITONAVIR,LOPINAVIR; RITONAVIR,KALETRA CAPS,KALETRA CAPS
2,1,2004-04-14,COMBIVIR cp enrobé,3466271,"1 Comprimé(s), Toutes les 12 heures pendant 30...",LAMIVUDINE; ZIDOVUDINE,J05AR01,1 Comprimé,Toutes les 12 heures,30 jour(s),150 mg; 300 mg,un comprimé,3466271.0,COMBIVIR,LAMIVUDINE; ZIDOVUDINE,LAMIVUDINE; ZIDOVUDINE,COMBIVIR CP ENROBÉ,COMBIVIR CP ENROBÉ
3,2,2013-10-01,TRUVADA 200MG/245MG CPR 30,3656563,1 comprimé par jour pendant 1 mois,EMTRICITABINE; TENOFOVIR DISOPROXIL,J05AR03,1 comprimé,par jour,1 mois,200 mg; 300 mg,un comprimé,3656563.0,TRUVADA,EMTRICITABINE; TENOFOVIR DISOPROXIL,EMTRICITABINE; TENOFOVIR DISOPROXIL,TRUVADA 200MG/245MG CPR 30,TRUVADA / CPR 30
4,2,2013-10-01,ISENTRESS 400MG CPR 60,3830848,2 comprimés par jour pendant 1 mois,RALTEGRAVIR POTASSIQUE,J05AJ01,2 comprimés,par jour,1 mois,"434,4 mg",un comprimé,3830848.0,ISENTRESS,RALTEGRAVIR POTASSIQUE,RALTEGRAVIR,ISENTRESS 400MG CPR 60,ISENTRESS CPR 60


In [131]:
df.to_excel("df.xlsx",index=False)

# Grouper dci 

## Étape 1 — Nettoyage du libellé

In [99]:
def clean_prelibt(s):
    return (
        str(s).upper()
              .replace("’", "'")
              .replace("-", " ")
              .strip()
    )

df["PRELIBT_NORM"] = df["Prelibt"].map(clean_prelibt)


## Étape 2 — Grouper les médicaments par DCI

In [97]:
grouped2 = (
    df.groupby("PRELIBT_NORM")["DCI_FINAL"]
      .agg(
          nb_medicaments="nunique",
          liste_DCI=lambda x: sorted(set(x.astype(str)))
      )
      .reset_index()
)


In [70]:
# grouped2 = (
#     df.groupby("PRELIBT_NORM")["DCI_FINALE"]
#       .agg(
#           nb_medicaments="nunique",
#           liste_DCI=lambda x: sorted(set(x))
#       )
#       .reset_index()
# )

In [102]:
# grouped2.to_excel("grouped2.xlsx")

In [98]:
grouped2

,PRELIBT_NORM,nb_medicaments,liste_DCI
0,(CARBOSYLANE),1,[CHARBON ACTIVE; SIMETICONE]
1,(CYTEAL),1,[CHLOROCRESOL;HEXAMIDINE; SOLUTION DE DIGLUCON...
2,(DEXERYL CR DERM 500G),0,[nan]
3,(DEXERYL),0,[nan]
4,(ELEVIT VITAMINE B9),1,[ACIDE FOLIQUE;ALPHA TOCOPHEROL; ASCORBATE DE ...
...,...,...,...
15973,ZYRTECSET 10MG CPR SECABLE 7 PR12,1,[CETIRIZINE]
15974,ZYTIGA 250MG CPR 120,1,[ABIRATERONE]
15975,ZYTIGA 500MG CPR 60,1,[ABIRATERONE]
15976,ZYVOXID 2MG/ML SOL PERF 300ML 10,1,[LINEZOLIDE]


## Étape 3 — Détection automatique des divergences

In [99]:
grouped2["divergence"] = grouped2["nb_medicaments"] > 1


In [100]:
suspects2 = grouped2[grouped2["divergence"]]


In [102]:
suspects2

,PRELIBT_NORM,nb_medicaments,liste_DCI,divergence


## Étape 4 — Mesurer la similarité entre dci

In [27]:
from difflib import SequenceMatcher
from itertools import combinations

def dci_similarity(dcis):
    if len(dcis) < 2:
        return 1.0
    sims = [
        SequenceMatcher(None, a, b).ratio()
        for a, b in combinations(dcis, 2)
    ]
    return max(sims)  # max, pas moyenne


In [112]:
grouped2["dci_similarity"] = grouped2["liste_DCI"].apply(dci_similarity)


In [113]:
def categorie(sim):
    if sim >= 0.9:
        return "typo_normalisation"
    elif sim >= 0.75:
        return "sel_forme"
    elif sim >= 0.5:
        return "combinaison_possible"
    else:
        return "erreur_grave"


In [114]:
grouped2["categorie"] = grouped2["dci_similarity"].apply(categorie)


In [34]:
grouped2[grouped2["categorie"]=="typo_normalisation"]

,PRELIBT_NORM,nb_medicaments,liste_DCI,divergence,dci_similarity,categorie
0,(CARBOSYLANE),1,[CHARBON ACTIVE; SIMETICONE],False,1.0,typo_normalisation
1,(CYTEAL),1,[CHLOROCRESOL;HEXAMIDINE; SOLUTION DE DIGLUCON...,False,1.0,typo_normalisation
2,(DEXERYL CR DERM 500G),0,[nan],False,1.0,typo_normalisation
3,(DEXERYL),0,[nan],False,1.0,typo_normalisation
4,(ELEVIT VITAMINE B9),1,[ACIDE FOLIQUE;ALPHA TOCOPHEROL; ASCORBATE DE ...,False,1.0,typo_normalisation
...,...,...,...,...,...,...
15973,ZYRTECSET 10MG CPR SECABLE 7 PR12,1,[CETIRIZINE],False,1.0,typo_normalisation
15974,ZYTIGA 250MG CPR 120,1,[ABIRATERONE],False,1.0,typo_normalisation
15975,ZYTIGA 500MG CPR 60,1,[ABIRATERONE],False,1.0,typo_normalisation
15976,ZYVOXID 2MG/ML SOL PERF 300ML 10,1,[LINEZOLIDE],False,1.0,typo_normalisation


In [115]:
from collections import Counter

def dci_majoritaire(dcis):
    return Counter(dcis).most_common(1)[0][0]

grouped2["DCI_CORRIGEE"] = grouped2["liste_DCI"].apply(dci_majoritaire)


In [129]:
grouped2[grouped2["categorie"] == "erreur_grave"].to_excel(
    "audit_erreurs_dci.xlsx", index=False
)


## pour la correction dci 

In [134]:
g=g.drop_duplicates(subset="DCI")
g.to_excel("grouped2_unique_dernier.xlsx",index=False)

In [104]:
grouped2.to_excel("grouped2_dernier.xlsx",index=False)


In [130]:
g["DCI"]=g["DCI_CORRIGEE"]

C:\Users\toufi\AppData\Local\Temp\ipykernel_9312\734818642.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  g["DCI"]=g["DCI_CORRIGEE"]


In [131]:
g=g[["PRELIBT_NORM","DCI_CORRIGEE","DCI"]]

In [133]:
g.columns = ['PRELIBT_NORM', 'DCI', 'DCI_CORRIGEE']

In [135]:
g

,PRELIBT_NORM,DCI,DCI_CORRIGEE
0,(CARBOSYLANE),CHARBON ACTIVE; SIMETICONE,CHARBON ACTIVE; SIMETICONE
1,(CYTEAL),CHLOROCRESOL;HEXAMIDINE; SOLUTION DE DIGLUCONA...,CHLOROCRESOL;HEXAMIDINE; SOLUTION DE DIGLUCONA...
2,(DEXERYL CR DERM 500G),nan,nan
4,(ELEVIT VITAMINE B9),ACIDE FOLIQUE;ALPHA TOCOPHEROL; ASCORBATE DE C...,ACIDE FOLIQUE;ALPHA TOCOPHEROL; ASCORBATE DE C...
5,(GELOX),ALUMINIUM; HYDROXYDE DE MAGNESIUM; MONMECTITE,ALUMINIUM; HYDROXYDE DE MAGNESIUM; MONMECTITE
...,...,...,...
15844,ZONEGRAN 100MG GELULE 56,ZONISAMIDE,ZONISAMIDE
15898,ZOSTAVAX PDR+SOL SER SC IM 1,VIRUS VARICELLE ZONA SOUCHE OKA/MERCK,VIRUS VARICELLE ZONA SOUCHE OKA/MERCK
15924,ZUTECTRA 500UI SOL SC SER 1ML 5,IMMUNOGLOBULINE HUMAINE DE L'HEPATITE B,IMMUNOGLOBULINE HUMAINE DE L'HEPATITE B
15925,ZYBAN LP 150 MG COMPRIMÉ À LIBÉRATION PROLONGÉ...,BUPROPIONE,BUPROPIONE


In [136]:
g1 = g[g['DCI'].astype(str).str.split().str.len() > 1].copy()


In [138]:
g1.to_excel("g1.xlsx",index=False)

# appliquer correctionn

## Étape 1 — Construire la table de correction

In [88]:
grouped2 = pd.read_excel("grouped2.xlsx")


In [76]:
grouped2.head(5)

,PRELIBT_NORM,nb_medicaments,liste_DCI,divergence,dci_similarity,categorie,DCI_CORRIGEE
0,(CARBOSYLANE),1,['CHARBON ACTIVE; SIMETICONE'],False,1.0,typo_normalisation,CHARBON ACTIVE; SIMETICONE
1,(CYTEAL),1,['CHLOROCRESOL;HEXAMIDINE; SOLUTION DE DIGLUCO...,False,1.0,typo_normalisation,CHLOROCRESOL;HEXAMIDINE; SOLUTION DE DIGLUCONA...
2,(DEXERYL CR DERM 500G),1,['<NA>'],False,1.0,typo_normalisation,NaN
3,(DEXERYL),1,['<NA>'],False,1.0,typo_normalisation,NaN
4,(ELEVIT VITAMINE B9),1,['ACIDE FOLIQUE;ALPHA TOCOPHEROL; ASCORBATE DE...,False,1.0,typo_normalisation,ACIDE FOLIQUE;ALPHA TOCOPHEROL; ASCORBATE DE C...


In [3]:
# corrections = grouped2[
#     grouped2["categorie"].isin(["typo_normalisation", "sel_forme"])
# ][["PRELIBT_NORM", "DCI_CORRIGEE"]]


In [94]:
corrections =grouped2[["PRELIBT_NORM","DCI_CORRIGEE"]]

In [93]:
corrections

,PRELIBT_NORM,DCI_CORRIGEE
0,(CARBOSYLANE),CHARBON ACTIVE; SIMETICONE
1,(CYTEAL),CHLOROCRESOL;HEXAMIDINE; SOLUTION DE DIGLUCONA...
2,(DEXERYL CR DERM 500G),NaN
3,(DEXERYL),NaN
4,(ELEVIT VITAMINE B9),ACIDE FOLIQUE;ALPHA TOCOPHEROL; ASCORBATE DE C...
...,...,...
15973,ZYRTECSET 10MG CPR SECABLE 7 PR12,CETIRIZINE
15974,ZYTIGA 250MG CPR 120,ABIRATERONE
15975,ZYTIGA 500MG CPR 60,ABIRATERONE
15976,ZYVOXID 2MG/ML SOL PERF 300ML 10,LINEZOLIDE


In [95]:
# Supprimer les espaces inutiles et mettre en majuscule dans les 2 df
df["PRELIBT_NORM_clean"] = df["PRELIBT_NORM"].str.strip().str.upper()
corrections["PRELIBT_NORM_clean"] = corrections["PRELIBT_NORM"].str.strip().str.upper()

# Créer le dictionnaire de correction
correction_dict = corrections.set_index("PRELIBT_NORM_clean")["DCI_CORRIGEE"].to_dict()

# Appliquer la correction
df["DCI_FINAL"] = df["PRELIBT_NORM_clean"].map(correction_dict).combine_first(df["DCI_MERE"])


C:\Users\toufi\AppData\Local\Temp\ipykernel_9312\1025833761.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  corrections["PRELIBT_NORM_clean"] = corrections["PRELIBT_NORM"].str.strip().str.upper()


In [80]:
# Supposons que ton df s'appelle df et ton df de correction df_correction

# On crée un dictionnaire à partir de df_correction
correction_dict = corrections.set_index("PRELIBT_NORM")["DCI_CORRIGEE"].to_dict()

# On applique la correction sur le df principal
df["DCI_FINAL"] = df["PRELIBT_NORM"].map(correction_dict).combine_first(df["DCI_MERE"])


## Étape 2 — Joindre les corrections au DataFrame initial

In [13]:
df = pd.read_excel("df.xlsx")

In [46]:
df = df.merge(
    corrections,
    on="PRELIBT_NORM",
    how="left"
)


In [47]:
df.shape

(1048575, 21)

## Étape 3 — Appliquer la correction (sans perdre l’original)

In [50]:
df["DCI_FINALE"] = df["DCI_CORRIGEE_y"].combine_first(df["DCI_MERE"])


## Étape 4 — Traçabilité (très important)

In [51]:
df = df.merge(
    grouped2[["PRELIBT_NORM", "categorie"]],
    on="PRELIBT_NORM",
    how="left"
)


## Étape 5 — Vérification rapide

In [66]:
df[df["DCI_CORRIGEE_y"].notna()][
    ["PRELIBT_NORM", "DCI_MERE", "DCI_CORRIGEE_y", "DCI_FINALE", "categorie"]
]


## Cas à NE PAS corriger automatiquement

In [ ]:
df.loc[
    df["categorie"].isin(["combinaison_possible", "erreur_grave"]),
    "DCI_FINALE"
] = df["DCI_MERE"]


In [106]:
df.columns

Index(['ID', 'Date', 'Prelibt', 'CIP', 'Prepost', 'DCI', 'ATC', 'DOSE', 'Freq',
       'Durée', 'dosage', 'dosage_nature', 'cip7', 'Med', 'DCI_NORM',
       'DCI_MERE', 'PRELIBT_NORM', 'PRELIBT_BASE', 'DCI_CORRIGEE_x',
       'DCI_FINALE', 'DCI_CORRIGEE_y', 'categorie', 'DCI_FINAL',
       'PRELIBT_NORM_clean'],
      dtype='object')

In [108]:
df1 = df[['ID', 'Date', 'Prelibt', 'CIP', 'Prepost', 'DCI', 'ATC', 'DOSE', 'Freq','Durée', 'dosage', 'dosage_nature', 'cip7', 'Med', 'DCI_NORM','DCI_MERE', 'PRELIBT_NORM', 'PRELIBT_BASE','DCI_FINAL']]

In [110]:
df1.to_excel("df1.xlsx",index=False)

# Binome

In [2]:
df = pd.read_excel("df1.xlsx")

In [15]:
nan=df[df["DCI_FINAL"].isna()]
nan=nan.drop_duplicates(subset="Prelibt")

In [18]:
nan=nan.drop_duplicates(subset="PRELIBT_BASE")

In [19]:
nan=nan[["ID","CIP","Prelibt","PRELIBT_BASE","DCI_FINAL"]]

In [22]:
nan.to_excel("Nan.xlsx",index=False)

In [68]:
I = df[["Prelibt","DCI_FINAL"]]
I["DCI_corrigee"]=I["DCI_FINAL"]

C:\Users\toufi\AppData\Local\Temp\ipykernel_12780\3877712600.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  I["DCI_corrigee"]=I["DCI_FINAL"]


In [69]:
I=I.drop_duplicates(subset="DCI_FINAL")

In [70]:
I

,Prelibt,DCI_FINAL,DCI_corrigee
0,BACTRIM cp Ad,SULFAMETHOXAZOLE; TRIMETHOPRIME,SULFAMETHOXAZOLE; TRIMETHOPRIME
1,KALETRA caps,LOPINAVIR; RITONAVIR,LOPINAVIR; RITONAVIR
2,COMBIVIR cp enrobé,LAMIVUDINE; ZIDOVUDINE,LAMIVUDINE; ZIDOVUDINE
3,TRUVADA 200MG/245MG CPR 30,EMTRICITABINE; TENOFOVIR DISOPROXIL,EMTRICITABINE; TENOFOVIR DISOPROXIL
4,ISENTRESS 400MG CPR 60,RALTEGRAVIR,RALTEGRAVIR
...,...,...,...
1012982,SEDATIF TIBER SP 150ML,AUBEPINE; POTASSIUM; SODIUM; PASSIFLORE,AUBEPINE; POTASSIUM; SODIUM; PASSIFLORE
1020273,INLYTA 5MG CPR 56,AXITINIB,AXITINIB
1020390,CABOMETYX 40MG CPR FL30,MALATE DE CABOZANTINIB,MALATE DE CABOZANTINIB
1039962,NUVARING 15MCG/120MCG/24H A.VAG 3,ETHINYLESTRADIOL; ETONOGESTREL,ETHINYLESTRADIOL; ETONOGESTREL


### flag tow 

In [71]:
# Fonction pour vérifier si au moins un élément contient 2 mots ou plus
def flag_two_words_or_more(dci):
    dci_clean = ';'.join([x.strip() for x in str(dci).split(';')])  # nettoyer espaces autour du ;
    for part in dci_clean.split(';'):
        if len(part.split()) >= 2:
            return 0  # on garde
    return 1  # on ne garde pas

# Créer la colonne
I['filter_flag'] = I['DCI_FINAL'].apply(flag_two_words_or_more)

I


,Prelibt,DCI_FINAL,DCI_corrigee,filter_flag
0,BACTRIM cp Ad,SULFAMETHOXAZOLE; TRIMETHOPRIME,SULFAMETHOXAZOLE; TRIMETHOPRIME,1
1,KALETRA caps,LOPINAVIR; RITONAVIR,LOPINAVIR; RITONAVIR,1
2,COMBIVIR cp enrobé,LAMIVUDINE; ZIDOVUDINE,LAMIVUDINE; ZIDOVUDINE,1
3,TRUVADA 200MG/245MG CPR 30,EMTRICITABINE; TENOFOVIR DISOPROXIL,EMTRICITABINE; TENOFOVIR DISOPROXIL,0
4,ISENTRESS 400MG CPR 60,RALTEGRAVIR,RALTEGRAVIR,1
...,...,...,...,...
1012982,SEDATIF TIBER SP 150ML,AUBEPINE; POTASSIUM; SODIUM; PASSIFLORE,AUBEPINE; POTASSIUM; SODIUM; PASSIFLORE,1
1020273,INLYTA 5MG CPR 56,AXITINIB,AXITINIB,1
1020390,CABOMETYX 40MG CPR FL30,MALATE DE CABOZANTINIB,MALATE DE CABOZANTINIB,0
1039962,NUVARING 15MCG/120MCG/24H A.VAG 3,ETHINYLESTRADIOL; ETONOGESTREL,ETHINYLESTRADIOL; ETONOGESTREL,1


In [72]:
I.to_excel("III.xlsx",index=False)

In [67]:
I

,Prelibt,DCI_FINAL,DCI_corrigee,filter_flag
0,BACTRIM cp Ad,SULFAMETHOXAZOLE; TRIMETHOPRIME,SULFAMETHOXAZOLE; TRIMETHOPRIME,1
1,KALETRA caps,LOPINAVIR; RITONAVIR,LOPINAVIR; RITONAVIR,1
2,COMBIVIR cp enrobé,LAMIVUDINE; ZIDOVUDINE,LAMIVUDINE; ZIDOVUDINE,1
3,TRUVADA 200MG/245MG CPR 30,EMTRICITABINE; TENOFOVIR DISOPROXIL,EMTRICITABINE; TENOFOVIR DISOPROXIL,0
4,ISENTRESS 400MG CPR 60,RALTEGRAVIR,RALTEGRAVIR,1
...,...,...,...,...
1012982,SEDATIF TIBER SP 150ML,AUBEPINE; POTASSIUM; SODIUM; PASSIFLORE,AUBEPINE; POTASSIUM; SODIUM; PASSIFLORE,1
1020273,INLYTA 5MG CPR 56,AXITINIB,AXITINIB,1
1020390,CABOMETYX 40MG CPR FL30,MALATE DE CABOZANTINIB,MALATE DE CABOZANTINIB,0
1039962,NUVARING 15MCG/120MCG/24H A.VAG 3,ETHINYLESTRADIOL; ETONOGESTREL,ETHINYLESTRADIOL; ETONOGESTREL,1


In [47]:
I1=I[I["filter_flag"]==1]

In [48]:
I1

,Prelibt,DCI_FINAL,DCI_corrigee,filter_flag
0,BACTRIM cp Ad,SULFAMETHOXAZOLE; TRIMETHOPRIME,SULFAMETHOXAZOLE; TRIMETHOPRIME,1
1,KALETRA caps,LOPINAVIR; RITONAVIR,LOPINAVIR; RITONAVIR,1
2,COMBIVIR cp enrobé,LAMIVUDINE; ZIDOVUDINE,LAMIVUDINE; ZIDOVUDINE,1
4,ISENTRESS 400MG CPR 60,RALTEGRAVIR,RALTEGRAVIR,1
5,UVEDOSE 100 000UI/2ML AMP BUV 1,CHOLECALCIFEROL,CHOLECALCIFEROL,1
...,...,...,...,...
1003729,Golimumab 100 mg Solution injectable,GOLIMUMAB,GOLIMUMAB,1
1012982,SEDATIF TIBER SP 150ML,AUBEPINE; POTASSIUM; SODIUM; PASSIFLORE,AUBEPINE; POTASSIUM; SODIUM; PASSIFLORE,1
1020273,INLYTA 5MG CPR 56,AXITINIB,AXITINIB,1
1039962,NUVARING 15MCG/120MCG/24H A.VAG 3,ETHINYLESTRADIOL; ETONOGESTREL,ETHINYLESTRADIOL; ETONOGESTREL,1


In [46]:
II=II.drop("filter_flag",axis=1)

II.to_excel("I.xlsx",index=False)

In [33]:
I2

,Prelibt,DCI_FINAL,DCI_corrigee
0,BACTRIM cp Ad,"[['SULFAMETHOXAZOLE', 'TRIMETHOPRIME']]",SULFAMETHOXAZOLE; TRIMETHOPRIME
1,KALETRA caps,"[['LOPINAVIR', 'RITONAVIR']]",LOPINAVIR; RITONAVIR
2,COMBIVIR cp enrobé,"[['LAMIVUDINE', 'ZIDOVUDINE']]",LAMIVUDINE; ZIDOVUDINE
3,TRUVADA 200MG/245MG CPR 30,"[['EMTRICITABINE', 'TENOFOVIR DISOPROXIL']]",EMTRICITABINE; TENOFOVIR DISOPROXIL
4,ISENTRESS 400MG CPR 60,[['RALTEGRAVIR']],RALTEGRAVIR
...,...,...,...
1012982,SEDATIF TIBER SP 150ML,"[['AUBEPINE', 'POTASSIUM', 'SODIUM', 'PASSIFLO...",AUBEPINE; POTASSIUM; SODIUM; PASSIFLORE
1020273,INLYTA 5MG CPR 56,[['AXITINIB']],AXITINIB
1020390,CABOMETYX 40MG CPR FL30,[['MALATE DE CABOZANTINIB']],MALATE DE CABOZANTINIB
1039962,NUVARING 15MCG/120MCG/24H A.VAG 3,"[['ETHINYLESTRADIOL', 'ETONOGESTREL']]",ETHINYLESTRADIOL; ETONOGESTREL


In [35]:
# Forcer toutes les valeurs en string et remplacer NaN
I.loc[:, 'DCI_FINAL'] = I['DCI_FINAL'].fillna('').astype(str)

# Séparer sur ';' et nettoyer les espaces
I.loc[:, 'DCI_FINAL'] = I['DCI_FINAL'].str.split(';').apply(lambda x: [i.strip() for i in x if i.strip() != ''])

# Exploser en lignes
Icar = I.explode('DCI_FINAL').reset_index(drop=True)



In [36]:
Icar

,Prelibt,DCI_FINAL,DCI_corrigee
0,BACTRIM cp Ad,"['[""[\'SULFAMETHOXAZOLE\', \'TRIMETHOPRIME\']""]']",SULFAMETHOXAZOLE; TRIMETHOPRIME
1,KALETRA caps,"['[""[\'LOPINAVIR\', \'RITONAVIR\']""]']",LOPINAVIR; RITONAVIR
2,COMBIVIR cp enrobé,"['[""[\'LAMIVUDINE\', \'ZIDOVUDINE\']""]']",LAMIVUDINE; ZIDOVUDINE
3,TRUVADA 200MG/245MG CPR 30,"['[""[\'EMTRICITABINE\', \'TENOFOVIR DISOPROXIL...",EMTRICITABINE; TENOFOVIR DISOPROXIL
4,ISENTRESS 400MG CPR 60,"['[""[\'RALTEGRAVIR\']""]']",RALTEGRAVIR
...,...,...,...
1509,SEDATIF TIBER SP 150ML,"['[""[\'AUBEPINE\', \'POTASSIUM\', \'SODIUM\', ...",AUBEPINE; POTASSIUM; SODIUM; PASSIFLORE
1510,INLYTA 5MG CPR 56,"['[""[\'AXITINIB\']""]']",AXITINIB
1511,CABOMETYX 40MG CPR FL30,"['[""[\'MALATE DE CABOZANTINIB\']""]']",MALATE DE CABOZANTINIB
1512,NUVARING 15MCG/120MCG/24H A.VAG 3,"['[""[\'ETHINYLESTRADIOL\', \'ETONOGESTREL\']""]']",ETHINYLESTRADIOL; ETONOGESTREL


In [37]:
I['DCI_corrigee'] = I['DCI_corrigee'].fillna('')  # remplacer les NaN par chaîne vide
I['DCI_corrigee'] = I['DCI_corrigee'].str.split(';').apply(lambda x: [i.strip() for i in x if i.strip() != ''])
df_exploded = I.explode('DCI_corrigee').reset_index(drop=True)


C:\Users\toufi\AppData\Local\Temp\ipykernel_12780\764352196.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  I['DCI_corrigee'] = I['DCI_corrigee'].fillna('')  # remplacer les NaN par chaîne vide
C:\Users\toufi\AppData\Local\Temp\ipykernel_12780\764352196.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  I['DCI_corrigee'] = I['DCI_corrigee'].str.split(';').apply(lambda x: [i.strip() for i in x if i.strip() != ''])


In [38]:
I=df_exploded

,Prelibt,DCI_FINAL,DCI_corrigee
0,BACTRIM cp Ad,"[['[""[\'SULFAMETHOXAZOLE\', \'TRIMETHOPRIME\']...",SULFAMETHOXAZOLE
1,BACTRIM cp Ad,"[['[""[\'SULFAMETHOXAZOLE\', \'TRIMETHOPRIME\']...",TRIMETHOPRIME
2,KALETRA caps,"[['[""[\'LOPINAVIR\', \'RITONAVIR\']""]']]",LOPINAVIR
3,KALETRA caps,"[['[""[\'LOPINAVIR\', \'RITONAVIR\']""]']]",RITONAVIR
4,COMBIVIR cp enrobé,"[['[""[\'LAMIVUDINE\', \'ZIDOVUDINE\']""]']]",LAMIVUDINE
...,...,...,...
2423,INLYTA 5MG CPR 56,"[['[""[\'AXITINIB\']""]']]",AXITINIB
2424,CABOMETYX 40MG CPR FL30,"[['[""[\'MALATE DE CABOZANTINIB\']""]']]",MALATE DE CABOZANTINIB
2425,NUVARING 15MCG/120MCG/24H A.VAG 3,"[['[""[\'ETHINYLESTRADIOL\', \'ETONOGESTREL\']""...",ETHINYLESTRADIOL
2426,NUVARING 15MCG/120MCG/24H A.VAG 3,"[['[""[\'ETHINYLESTRADIOL\', \'ETONOGESTREL\']""...",ETONOGESTREL


# Grouper med

## Étape 1 — Nettoyage du libellé

In [35]:
def clean_prelibt(s):
    return (
        str(s).upper()
              .replace("’", "'")
              .replace("-", " ")
              .strip()
    )

df["PRELIBT_NORM"] = df["Prelibt"].map(clean_prelibt)


## Étape 2 — Grouper les médicaments par DCI

In [49]:
grouped = (
    df.groupby("DCI_FINAL")["PRELIBT_NORM"]
      .agg(
          nb_medicaments="nunique",
          liste_prelibt=lambda x: sorted(set(x))
      )
      .reset_index()
)


In [50]:
grouped

,DCI_FINAL,nb_medicaments,liste_prelibt
0,ABACAVIR,14,"[ABACAVIR 20 MG/ML SOLUTION BUVABLE, ABACAVIR ..."
1,ABACAVIR ; LAMIVUDINE,8,[ABACAVIR + LAMIVUDINE 600 MG/300 MG COMPRIMÉ ...
2,ABACAVIR; LAMIVUDINE,12,"[ABACAVIR/LAMI 600MG/300MG BGR CP30, ABACAVIR/..."
3,ABIRATERONE,3,"[ABIRATÉRONE 500 MG COMPRIMÉ, ZYTIGA 250MG CPR..."
4,ACAMPROSATE,6,[ACAMPROSATE 333 MG COMPRIMÉ ENROBÉ GASTRORÉSI...
...,...,...,...
1508,ZOLMITRIPTAN,14,"[ZOLMITRIPTAN 2,5 MG COMPRIMÉ ORODISPERSIBLE, ..."
1509,ZOLPIDEM,34,[STILNOX 10 MG COMPRIMÉ PELLICULÉ SÉCABLE BOÎT...
1510,ZONISAMIDE,3,"[ZONEGRAN 100MG GELULE 56, ZONEGRAN 25MG GELUL..."
1511,ZOPICLONE,50,"[IMOVANE 3,75 MG COMPRIMÉ PELLICULÉ BOÎTE DE 1..."


## Étape 3 — Détection automatique des divergences

In [51]:
grouped["divergence"] = grouped["nb_medicaments"] > 1


In [52]:
suspects = grouped[grouped["divergence"]]


In [53]:
suspects

,DCI_FINAL,nb_medicaments,liste_prelibt,divergence
0,ABACAVIR,14,"[ABACAVIR 20 MG/ML SOLUTION BUVABLE, ABACAVIR ...",True
1,ABACAVIR ; LAMIVUDINE,8,[ABACAVIR + LAMIVUDINE 600 MG/300 MG COMPRIMÉ ...,True
2,ABACAVIR; LAMIVUDINE,12,"[ABACAVIR/LAMI 600MG/300MG BGR CP30, ABACAVIR/...",True
3,ABIRATERONE,3,"[ABIRATÉRONE 500 MG COMPRIMÉ, ZYTIGA 250MG CPR...",True
4,ACAMPROSATE,6,[ACAMPROSATE 333 MG COMPRIMÉ ENROBÉ GASTRORÉSI...,True
...,...,...,...,...
1508,ZOLMITRIPTAN,14,"[ZOLMITRIPTAN 2,5 MG COMPRIMÉ ORODISPERSIBLE, ...",True
1509,ZOLPIDEM,34,[STILNOX 10 MG COMPRIMÉ PELLICULÉ SÉCABLE BOÎT...,True
1510,ZONISAMIDE,3,"[ZONEGRAN 100MG GELULE 56, ZONEGRAN 25MG GELUL...",True
1511,ZOPICLONE,50,"[IMOVANE 3,75 MG COMPRIMÉ PELLICULÉ BOÎTE DE 1...",True


## Étape 4 — Mesurer la similarité entre libellés 

In [41]:
import re

def strip_dosage(s):
    s = re.sub(r"\b\d+(\.\d+)?\s*(MG|G|ML|MCG|UI)\b", "", s)
    s = re.sub(r"\b(COMPRIME|GELULE|SOLUTION|SIROP)\b", "", s)
    return re.sub(r"\s{2,}", " ", s).strip()


In [42]:
df["PRELIBT_BASE"] = df["PRELIBT_NORM"].map(strip_dosage)


In [54]:
from difflib import SequenceMatcher
from itertools import combinations

def avg_similarity(values):
    if len(values) < 2:
        return 1.0
    sims = [
        SequenceMatcher(None, a, b).ratio()
        for a, b in combinations(values, 2)
    ]
    return sum(sims) / len(sims)


In [56]:
sim = (
    df.groupby("DCI_FINAL")["PRELIBT_BASE"]
      .apply(lambda x: avg_similarity(set(x)))
      .reset_index(name="similarite_moyenne")
)


In [57]:
sim

,DCI_FINAL,similarite_moyenne
0,ABACAVIR,0.426243
1,ABACAVIR ; LAMIVUDINE,0.648610
2,ABACAVIR; LAMIVUDINE,0.433888
3,ABIRATERONE,0.495346
4,ACAMPROSATE,0.470219
...,...,...
1508,ZOLMITRIPTAN,0.568922
1509,ZOLPIDEM,0.599523
1510,ZONISAMIDE,0.818182
1511,ZOPICLONE,0.595145


## Étape 5 — Identifier les incohérences

In [58]:
analysis = grouped.merge(sim, on="DCI_FINAL")


In [59]:
analysis["incoherent"] = analysis["similarite_moyenne"] < 0.75


In [60]:
analysis["score_coherence"] = (
    analysis["similarite_moyenne"]
    * (1 / analysis["nb_medicaments"])
)


In [61]:
analysis

,DCI_FINAL,nb_medicaments,liste_prelibt,divergence,similarite_moyenne,incoherent,score_coherence
0,ABACAVIR,14,"[ABACAVIR 20 MG/ML SOLUTION BUVABLE, ABACAVIR ...",True,0.426243,True,0.030446
1,ABACAVIR ; LAMIVUDINE,8,[ABACAVIR + LAMIVUDINE 600 MG/300 MG COMPRIMÉ ...,True,0.648610,True,0.081076
2,ABACAVIR; LAMIVUDINE,12,"[ABACAVIR/LAMI 600MG/300MG BGR CP30, ABACAVIR/...",True,0.433888,True,0.036157
3,ABIRATERONE,3,"[ABIRATÉRONE 500 MG COMPRIMÉ, ZYTIGA 250MG CPR...",True,0.495346,True,0.165115
4,ACAMPROSATE,6,[ACAMPROSATE 333 MG COMPRIMÉ ENROBÉ GASTRORÉSI...,True,0.470219,True,0.078370
...,...,...,...,...,...,...,...
1508,ZOLMITRIPTAN,14,"[ZOLMITRIPTAN 2,5 MG COMPRIMÉ ORODISPERSIBLE, ...",True,0.568922,True,0.040637
1509,ZOLPIDEM,34,[STILNOX 10 MG COMPRIMÉ PELLICULÉ SÉCABLE BOÎT...,True,0.599523,True,0.017633
1510,ZONISAMIDE,3,"[ZONEGRAN 100MG GELULE 56, ZONEGRAN 25MG GELUL...",True,0.818182,False,0.272727
1511,ZOPICLONE,50,"[IMOVANE 3,75 MG COMPRIMÉ PELLICULÉ BOÎTE DE 1...",True,0.595145,True,0.011903


In [65]:
analysis.sort_values("similarite_moyenne").head(20)


,DCI_FINAL,nb_medicaments,liste_prelibt,divergence,similarite_moyenne,incoherent,score_coherence
328,CALCITRIOL,2,"[ROCALTROL 0,25MCG CAPS 30, SILKIS 3MCG/G POM ...",True,0.162162,True,0.081081
1207,POLYOSIDE CAPSULAIRE VI DE SALMONELLA TYPHI; V...,2,"[TYAVAX SER 1ML 1, VACCIN FIÈVRE TYPHOÏDE + HÉ...",True,0.170213,True,0.085106
588,DOXORUBICINE,2,"[CAELYX SOL À DILUER P PERF IV 2 MG/ML, DOXORU...",True,0.175439,True,0.087719
1161,PEGINTERFERON BETA 1A,2,[PEGINTERFÉRON BÊTA 1A 125 MICROGRAMMES SOLUTI...,True,0.195122,True,0.097561
1062,MYCOPHENOLATE,3,[ACIDE MYCOPHÉNOLIQUE 360 MG COMPRIMÉ GASTRORÉ...,True,0.222222,True,0.074074
1356,SONIDEGIB,2,"[ODOMZO 200MG GELULE 30X1, SONIDEGIB 200 MG GÉ...",True,0.222222,True,0.111111
53,ACIDE SALICYLIQUE,3,"[CIELLA 0,1% SOL LAV OPHT DOSE 20, CORICIDE LE...",True,0.227886,True,0.075962
543,DICHLORHYDRATE DE ZUCLOPENTHIXOL,2,"[CLOPIXOL 2% SOL BUV 20ML, ZUCLOPENTHIXOL 2 % ...",True,0.238806,True,0.119403
686,EXTRAIT SEC RAFFINE ET QUANTIFIE DE FEUILLE DE...,2,"[GINKGO 250MG ARKOG GELUL 150, TANAKAN SOL BUV...",True,0.243902,True,0.121951
1393,TESTOSTERONE,3,"[ANDROGEL 25MG GEL SACHET 30, ANDROGEL 50MG GE...",True,0.250000,True,0.083333


In [63]:
analysis[analysis["incoherent"]==True]

,DCI_FINAL,nb_medicaments,liste_prelibt,divergence,similarite_moyenne,incoherent,score_coherence
0,ABACAVIR,14,"[ABACAVIR 20 MG/ML SOLUTION BUVABLE, ABACAVIR ...",True,0.426243,True,0.030446
1,ABACAVIR ; LAMIVUDINE,8,[ABACAVIR + LAMIVUDINE 600 MG/300 MG COMPRIMÉ ...,True,0.648610,True,0.081076
2,ABACAVIR; LAMIVUDINE,12,"[ABACAVIR/LAMI 600MG/300MG BGR CP30, ABACAVIR/...",True,0.433888,True,0.036157
3,ABIRATERONE,3,"[ABIRATÉRONE 500 MG COMPRIMÉ, ZYTIGA 250MG CPR...",True,0.495346,True,0.165115
4,ACAMPROSATE,6,[ACAMPROSATE 333 MG COMPRIMÉ ENROBÉ GASTRORÉSI...,True,0.470219,True,0.078370
...,...,...,...,...,...,...,...
1505,ZIDOVUDINE,13,"[RETROVIR 100MG GELULE 100, RETROVIR 100MG GEL...",True,0.563837,True,0.043372
1508,ZOLMITRIPTAN,14,"[ZOLMITRIPTAN 2,5 MG COMPRIMÉ ORODISPERSIBLE, ...",True,0.568922,True,0.040637
1509,ZOLPIDEM,34,[STILNOX 10 MG COMPRIMÉ PELLICULÉ SÉCABLE BOÎT...,True,0.599523,True,0.017633
1511,ZOPICLONE,50,"[IMOVANE 3,75 MG COMPRIMÉ PELLICULÉ BOÎTE DE 1...",True,0.595145,True,0.011903


In [66]:
analysis.to_excel("analysis.xlsx")